This file converts an svg file into a gcode format the cable robot can read.  In particular, it converts it to just polylines.

In the future, maybe we can switch to a spline representation, but for now it's polylines.

In [ ]:
import svgpathtools
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import dataclasses

In [ ]:
# infile = Path('files/brush_path_test_0107_2024.svg')
infile = Path('files/Library_test_0110.svg')
outfile = infile.with_suffix('.nc')
print(infile, '->', outfile)

In [ ]:
# First reorder the svg
paths, attributes = svgpathtools.svg2paths(infile)
if False:
    new_paths = [paths.pop()]
    while paths:
        end = new_paths[-1].end
        closest = min(paths, key=lambda p: abs(p.start - end))
        new_paths.append(closest)
        paths.remove(closest)

    svgpathtools.disvg(new_paths, filename=infile.with_stem(infile.stem + '_reordered').as_posix(), attributes=attributes)
    paths = new_paths

In [ ]:
def pt_to_mm(pt):
    return pt / 72 * 25.4

In [ ]:
# First convert to numpy polyline
POINT_EVERY_N_MM = 20

polylines = []

for path in paths:
    L = path.length()
    ts = np.arange(0, 1, POINT_EVERY_N_MM * 0.8 / pt_to_mm(L))
    ts = np.append(ts, 1)
    ps = np.array([path.point(t) for t in ts])
    ps = np.conjugate(pt_to_mm(ps))
    polylines.append(np.stack([np.real(ps), np.imag(ps)], axis=1) * 1e-3)
    # Sanity check
    # print(np.mean(np.abs(np.diff(ps))), '\t', np.max(np.abs(np.diff(ps[:-1])) - POINT_EVERY_N_MM))
    print(f'{np.max(np.abs(np.diff(ps[:-1]))):.2f}', end='\t')
    assert np.all(np.abs(np.diff(ps[:-1])) - POINT_EVERY_N_MM < 0.5)
# Sanity check
for polyline in polylines:
    # print(np.mean(np.linalg.norm(np.diff(polyline, axis=0), axis=1)))
    assert np.all(np.linalg.norm(np.diff(polyline, axis=0), axis=1)*1000 - POINT_EVERY_N_MM < 0.5)

In [ ]:
def plot_fig(polylines, kwargs={}):
    POINT_EVERY_N_MM = 10
    for polyline in polylines:
        plt.plot(*polyline.T, '--', linewidth=0.5, **kwargs)
    plt.axis('equal')
plot_fig(polylines)

In [ ]:
inch2m = lambda x: x * 25.4 / 1000
X_OFFSET = (1.887 - 0.037) + inch2m(- 3 - 72 / 2 + 0.5) + 60e-3
Y_OFFSET = 5.417 - 4.614 + 60e-3
X1 = (1.887 - 0.037) - inch2m(3 / 2)  # between left and center panes
X2 = (1.887 + inch2m(84) - 0.037) + inch2m(3 / 2)  # between center and right panes
YMID = (Y_OFFSET - 60e-3 + inch2m(71.5)) + inch2m(2.5 / 2) # between top and bottom panes

# Need to convert canvas coordinates into cable robot "carriage" coordinates
cdprrestTbrush = np.array([21.0e-2, 38.5e-2]) - np.array([0.5334, 0.381]) / 2

In [126]:
# Sanity checking!
@dataclasses.dataclass
class Bounds:
    xmin: float
    xmax: float
    ymin: float
    ymax: float
    def inset(self, amt):
        return Bounds(self.xmin + amt, self.xmax - amt, self.ymin + amt, self.ymax - amt)
    def translate(self, x, y):
        return Bounds(self.xmin + x, self.xmax + x, self.ymin + y, self.ymax + y)
    def __repr__(self):
        return f'Bounds{{X: [{self.xmin:.5f}, {self.xmax:.5f}], Y: [{self.ymin:.5f}, {self.ymax:.5f}]}}'
    def xmid(self):
        return (self.xmin + self.xmax) / 2
    def ymid(self):
        return (self.ymin + self.ymax) / 2
    def size(self):
        return (self.xmax - self.xmin, self.ymax - self.ymin)
    def to_cdpr_lims(self):
        return self.inset(60e-3).translate(*-cdprrestTbrush)
    def filter(self, polylines, include_border=True, margin=inch2m(3/2)):
        if margin != 0:
            return self.inset(-margin).filter(polylines, include_border=include_border, margin=0)
        if include_border:
            cond = lambda pl: np.all((self.xmin <= pl[:,0]) & (pl[:,0] <= self.xmax) & (self.ymin <= pl[:,1]) & (pl[:,1] <= self.ymax))
        else:
            cond = lambda pl: np.all((self.xmin < pl[:,0]) & (pl[:,0] < self.xmax) & (self.ymin < pl[:,1]) & (pl[:,1] < self.ymax))
        return list(filter(cond, polylines))

BRUSH_R = (3.1e-2 + 0.8e-2) / 2
BOUNDS_BOTTOM = Bounds(X1 + inch2m(3/2), X2 - inch2m(3/2), Y_OFFSET - 60e-3, YMID - inch2m(2.5/2))
BOUNDS_TOP = Bounds(X1 + inch2m(3/2), X2 - inch2m(3/2), YMID + inch2m(2.5/2), YMID + inch2m(2.5/2) + inch2m(71.5))
BOUNDS_LEFT = Bounds(X1 - inch2m(3/2+35.5), X1-inch2m(3/2), BOUNDS_BOTTOM.ymid() + inch2m(2.5/2), BOUNDS_TOP.ymid() - inch2m(2.5/2))
BOUNDS_RIGHT = Bounds(X2 + inch2m(3/2), X2 + inch2m(3/2+35.5), BOUNDS_BOTTOM.ymid() + inch2m(2.5/2), BOUNDS_TOP.ymid() - inch2m(2.5/2))
NAMES = ['Left', 'Right', 'Bottom', 'Top']
PANE_BOUNDS = [BOUNDS_LEFT, BOUNDS_RIGHT, BOUNDS_BOTTOM, BOUNDS_TOP]

print('Corner touch locs [BL-coords]:   ', BOUNDS_BOTTOM.inset(BRUSH_R).translate(-21.0e-2, -38.5e-2))
print('Corner touch locs [cdpr-coords]: ', BOUNDS_BOTTOM.inset(BRUSH_R).translate(*-cdprrestTbrush))
print('60mm buffer locs  [cdpr-coords]: ', BOUNDS_BOTTOM.inset(60e-3).translate(*-cdprrestTbrush))
print()
for name, bounds in zip(NAMES, PANE_BOUNDS):
    print(f'{name:6} 60mm buffer: {bounds.to_cdpr_lims()}')
print()
for name, bounds in zip(NAMES, PANE_BOUNDS):
    print(f'Size {name.lower():6} 60mm buffer: {bounds.to_cdpr_lims().size()}')

Corner touch locs [BL-coords]:    Bounds{X: [1.65950, 3.75410], Y: [0.43750, 2.21460]}
Corner touch locs [cdpr-coords]:  Bounds{X: [1.92620, 4.02080], Y: [0.62800, 2.40510]}
60mm buffer locs  [cdpr-coords]:  Bounds{X: [1.96670, 3.98030], Y: [0.66850, 2.36460]}

Left   60mm buffer: Bounds{X: [0.98880, 1.77050], Y: [1.60830, 3.30440]}
Right  60mm buffer: Bounds{X: [4.17650, 4.95820], Y: [1.60830, 3.30440]}
Bottom 60mm buffer: Bounds{X: [1.96670, 3.98030], Y: [0.66850, 2.36460]}
Top    60mm buffer: Bounds{X: [1.96670, 3.98030], Y: [2.54810, 4.24420]}

Size left   60mm buffer: (0.7816999999999998, 1.6960999999999995)
Size right  60mm buffer: (0.7817000000000007, 1.6960999999999995)
Size bottom 60mm buffer: (2.0136000000000003, 1.6960999999999995)
Size top    60mm buffer: (2.0136000000000003, 1.6961000000000004)


In [ ]:
# Rescale and translate
offset = np.min(np.concatenate(polylines, axis=0), axis=0) - np.array([X_OFFSET, Y_OFFSET])
polylines = [polyline - offset for polyline in polylines]
# Separate the different panes
panes = []
for BOUNDS_PANE in [BOUNDS_LEFT, BOUNDS_RIGHT, BOUNDS_BOTTOM, BOUNDS_TOP]:
    panes.append(BOUNDS_PANE.filter(polylines))
assert sum(len(pane) for pane in panes) == len(polylines), 'some polylines not in any pane'

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
for pane, c in zip(panes, ['k', 'b', 'g', 'y']):
    plot_fig(pane, {'color': c})
plt.vlines([X1, X2], 0.8, 4.5, linewidth=0.5, color='r')
plt.hlines(YMID, 0.8, 5.0, linewidth=0.5, color='r')

# Remove outer rectangles
def remove_outer_rectangle(polylines):
    bl = np.min(np.concatenate(polylines, axis=0), axis=0)
    return [polyline for polyline in polylines if not np.any(np.all(polyline < bl + 5e-3, axis=1), axis=0)]
def just_outer_rectangle(polylines):
    bl = np.min(np.concatenate(polylines, axis=0), axis=0)
    return [polyline for polyline in polylines if np.any(np.all(polyline < bl + 5e-3, axis=1), axis=0)]
panes_inner = [remove_outer_rectangle(pane) for pane in panes]
panes_outer = [just_outer_rectangle(pane) for pane in panes]
for pane_inner, pane_outer, pane in zip(panes_inner, panes_outer, panes):
    assert len(pane_inner) + 1 == len(pane), 'Outer rectangle removal error'
    assert len(pane_outer) == 1, 'Outer rectangle removal error'

plt.subplot(1, 2, 2)
for pane, c in zip(panes_inner, ['k', 'b', 'g', 'y']):
    plot_fig(pane, {'color': c})

In [127]:
# Print out bounds, for sanity checking
print('Pane artwork bounds in BL coordinates:')
for name, pane_outer in zip(NAMES, panes_outer):
    xmin, ymin = np.min([np.min(polyline, axis=0) for polyline in pane_outer], axis=0)
    xmax, ymax = np.max([np.max(polyline, axis=0) for polyline in pane_outer], axis=0)
    print(f'  {name:6}: [{xmin:.5f}, {xmax:.5f}], [{ymin:.5f}, {ymax:.5f}]')
print('Pane artwork bounds in cdpr coordinates:')
# Translate to cdpr coordinates
cdpr_panes_inner = [[-cdprrestTbrush + polyline for polyline in pane] for pane in panes_inner]
cdpr_panes_outer = [[-cdprrestTbrush + polyline for polyline in pane] for pane in panes_outer]
for name, pane_outer in zip(NAMES, cdpr_panes_outer):
    xmin, ymin = np.min([np.min(polyline, axis=0) for polyline in pane_outer], axis=0)
    xmax, ymax = np.max([np.max(polyline, axis=0) for polyline in pane_outer], axis=0)
    print(f'  {name:6}: [{xmin:.5f}, {xmax:.5f}], [{ymin:.5f}, {ymax:.5f}]')

# Print boundaries for reference
print('Expected pane bounds (with 60mm buffer) in cdpr coordinates:')
for name, bounds in zip(NAMES, PANE_BOUNDS):
    print(f'  {name:6}: {bounds.to_cdpr_lims()}')
# Print boundaries for reference
print('Difference:')
for name, bounds, pane_outer in zip(NAMES, PANE_BOUNDS, cdpr_panes_outer):
    xmin, ymin = np.min([np.min(polyline, axis=0) for polyline in pane_outer], axis=0)
    xmax, ymax = np.max([np.max(polyline, axis=0) for polyline in pane_outer], axis=0)
    expected = bounds.to_cdpr_lims()
    print(f'  {name:6}: [{xmin-expected.xmin:.5f}, {xmax-expected.xmax:.5f}], [{ymin-expected.ymin:.5f}, {ymax-expected.ymax:.5f}]')

Pane artwork bounds in BL coordinates:
  Left  : [0.93210, 1.71380], [1.73930, 3.43540]
  Right : [4.11980, 4.90150], [1.73930, 3.43540]
  Bottom: [1.91000, 3.92360], [0.86300, 2.55910]
  Top   : [1.91000, 3.92360], [2.74260, 4.43870]
Pane artwork bounds in cdpr coordinates:
  Left  : [0.98880, 1.77050], [1.54480, 3.24090]
  Right : [4.17650, 4.95820], [1.54480, 3.24090]
  Bottom: [1.96670, 3.98030], [0.66850, 2.36460]
  Top   : [1.96670, 3.98030], [2.54810, 4.24420]
Expected pane bounds (with 60mm buffer) in cdpr coordinates:
  Left  : Bounds{X: [0.98880, 1.77050], Y: [1.60830, 3.30440]}
  Right : Bounds{X: [4.17650, 4.95820], Y: [1.60830, 3.30440]}
  Bottom: Bounds{X: [1.96670, 3.98030], Y: [0.66850, 2.36460]}
  Top   : Bounds{X: [1.96670, 3.98030], Y: [2.54810, 4.24420]}
Difference:
  Left  : [0.00000, -0.00000], [-0.06350, -0.06350]
  Right : [0.00000, 0.00000], [-0.06350, -0.06350]
  Bottom: [-0.00000, 0.00000], [0.00000, 0.00000]
  Top   : [-0.00000, 0.00000], [0.00000, 0.00000]


In [ ]:
# Export panes
for fname, pl in zip(NAMES, cdpr_panes_inner):
    outfile_ = outfile.with_stem(outfile.stem + '_' + fname.lower())
    with open(outfile_, 'w') as f:
        for polyline in pl:
            xs, ys = polyline.T * 1e3
            # Write path to file
            f.write(f'G0 X{xs[0]:.3f} Y{ys[0]:.3f}\n')
            for x, y in zip(xs, ys):
                f.write(f'G1 X{x:.3f} Y{y:.3f}\n')
            f.write(f'G0 X{xs[-1]:.3f} Y{ys[-1]:.3f}\n')